In [ ]:
from google.colab import drive as MountDrive
MountDrive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


In [ ]:
worksheet = gc.open('covid_data').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
col = rows[0]
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,Date,state,tot_cases,conf_cases,prob_cases,new_case,pnew_case,tot_death,conf_death,prob_death,new_death,pnew_death,Administered,Admin_Per_100K,Administered_Dose1_Recip,Series_Complete_Yes
1,1/12/2021,Alabama,4.09E+05,3.29E+05,7.97E+04,3.86E+03,1.20E+03,9.05E+03,7.67E+03,1.37E+03,1.04E+02,4.90E+01,7.81E+04,1.59E+03,0.00E+00,0.00E+00
2,1/13/2021,Alabama,4.13E+05,3.32E+05,8.07E+04,3.85E+03,1.07E+03,9.17E+03,7.74E+03,1.44E+03,1.27E+02,6.60E+01,8.40E+04,1.71E+03,0.00E+00,0.00E+00
3,1/14/2021,Alabama,4.16E+05,3.34E+05,8.16E+04,3.22E+03,8.22E+02,9.25E+03,7.78E+03,1.47E+03,7.70E+01,3.00E+01,9.23E+04,1.88E+03,0.00E+00,0.00E+00
4,1/15/2021,Alabama,4.19E+05,3.37E+05,8.24E+04,3.19E+03,8.01E+02,9.33E+03,7.83E+03,1.50E+03,8.00E+01,2.80E+01,1.01E+05,2.05E+03,0.00E+00,0.00E+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15646,11/16/2021,Wyoming,1.08E+05,8.62E+04,2.23E+04,3.10E+02,1.17E+02,1.35E+03,1.35E+03,0.00E+00,4.90E+01,0.00E+00,6.09E+05,1.05E+05,3.04E+05,2.60E+05
15647,11/17/2021,Wyoming,1.09E+05,8.63E+04,2.24E+04,2.45E+02,1.11E+02,1.35E+03,1.35E+03,0.00E+00,0.00E+00,0.00E+00,6.10E+05,1.05E+05,3.04E+05,2.60E+05
15648,11/18/2021,Wyoming,1.09E+05,8.66E+04,2.25E+04,4.25E+02,8.90E+01,1.35E+03,1.35E+03,0.00E+00,0.00E+00,0.00E+00,6.14E+05,1.06E+05,3.05E+05,2.61E+05
15649,11/19/2021,Wyoming,1.09E+05,8.68E+04,2.25E+04,2.35E+02,6.70E+01,1.35E+03,1.35E+03,0.00E+00,0.00E+00,0.00E+00,6.16E+05,1.06E+05,3.05E+05,2.61E+05


In [ ]:
import numpy as np

In [ ]:
data = np.array(rows)
data.shape

(15651, 16)

In [ ]:
#removing column types
columns = data[0,:]
data = data[1:,:]
columns

array(['Date', 'state', 'tot_cases', 'conf_cases', 'prob_cases',
       'new_case', 'pnew_case', 'tot_death', 'conf_death', 'prob_death',
       'new_death', 'pnew_death', 'Administered', 'Admin_Per_100K',
       'Administered_Dose1_Recip', 'Series_Complete_Yes'], dtype='<U24')

In [ ]:
#removing dates and state names
date_names = data[:,0:2]
data = data[:,2:]
date_names

array([['1/12/2021', 'Alabama'],
       ['1/13/2021', 'Alabama'],
       ['1/14/2021', 'Alabama'],
       ...,
       ['11/18/2021', 'Wyoming'],
       ['11/19/2021', 'Wyoming'],
       ['11/20/2021', 'Wyoming']], dtype='<U24')

In [ ]:
states = 50
days = int(data.shape[0]/states) #for 50 states
days

313

In [ ]:
#reshape data to following: (days,states,features)
data = data.reshape(days,states,-1)

#rotate axis to have : (states,days,features)
data = np.swapaxes(data,0,1)
data = data.astype('float64')
data.shape

(50, 313, 14)

In [ ]:
#get temperature data
worksheet = gc.open('state_temp').sheet1

# get_all_values gives a list of rows.
temperature = worksheet.get_all_values()
col = temperature[0]
print(temperature)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)


[['Date', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-16', '2021-01-17', '2021-01-18', '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-23', '2021-01-24', '2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29', '2021-01-30', '2021-01-31', '2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-13', '2021-02-14', '2021-02-15', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', '2021-02-20', '2021-02-21', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-02-27', '2021-02-28', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-06', '2021-03-07', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021-03-13', '2021-03-14', '2021-03-15', '2021-03-16', '2021-03-17', '2021-03-18', '2021-03-19', '2021-03-20', '2021-03-21', '2021-03-22', '2021-03-2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313
0,Date,2021-01-12,2021-01-13,2021-01-14,2021-01-15,2021-01-16,2021-01-17,2021-01-18,2021-01-19,2021-01-20,2021-01-21,2021-01-22,2021-01-23,2021-01-24,2021-01-25,2021-01-26,2021-01-27,2021-01-28,2021-01-29,2021-01-30,2021-01-31,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,2021-02-11,2021-02-12,2021-02-13,2021-02-14,2021-02-15,2021-02-16,2021-02-17,2021-02-18,2021-02-19,...,2021-10-12,2021-10-13,2021-10-14,2021-10-15,2021-10-16,2021-10-17,2021-10-18,2021-10-19,2021-10-20,2021-10-21,2021-10-22,2021-10-23,2021-10-24,2021-10-25,2021-10-26,2021-10-27,2021-10-28,2021-10-29,2021-10-30,2021-10-31,2021-11-01,2021-11-02,2021-11-03,2021-11-04,2021-11-05,2021-11-06,2021-11-07,2021-11-08,2021-11-09,2021-11-10,2021-11-11,2021-11-12,2021-11-13,2021-11-14,2021-11-15,2021-11-16,2021-11-17,2021-11-18,2021-11-19,2021-11-20
1,Alabama,35.04166667,35.93055556,42.51388889,44.36111111,39.52777778,42.44444444,41.94444444,43.95833333,47.40277778,50.36111111,49.41666667,47.55555556,52.76388889,64.65277778,61.36111111,51.25,40.63888889,42.20833333,46.59722222,55.19444444,38.75,38.875,38.83333333,43.86111111,44.76388889,40.66666667,40.05555556,45.66666667,55.68055556,56.45833333,53.375,42.875,39,39.25,35.29166667,23.73611111,34,40.31944444,37.08333333,...,73.86111111,73.86111111,73.01388889,73.54166667,59.70833333,56.20833333,56.83333333,59.36111111,65.93055556,70.08333333,63.125,62.125,63.91666667,68.73611111,59.55555556,61,62.27777778,54.30555556,54.98611111,57.79166667,57.26388889,56.19444444,51.44444444,50.16666667,50.05555556,49.09722222,50.95833333,54.58333333,56.29166667,58.61111111,60.61111111,55.27777778,43.59722222,45.95833333,49.86111111,58.15277778,61.625,55.63888889,46.97222222,48.875
2,Alaska,27.41666667,26.33333333,29.58333333,29,28.33333333,35,35.83333333,30.5,28.41666667,30.08333333,33.08333333,32.91666667,29.75,21.58333333,12.66666667,9.416666667,18.33333333,18.83333333,14.33333333,8.083333333,4.083333333,4,11.16666667,14.08333333,11.41666667,10.5,5.833333333,1.916666667,2.916666667,15.91666667,21.5,18.08333333,10.33333333,12.91666667,18.41666667,23.33333333,27,26.08333333,12.66666667,...,37.83333333,34.66666667,34.25,33.08333333,32.58333333,34.16666667,36.83333333,38.33333333,39.25,37.66666667,35.75,39.75,39.83333333,34,31.08333333,29.33333333,29.5,30.08333333,38.58333333,42.58333333,39.25,39.08333333,39.75,38.91666667,35.41666667,33.91666667,31.41666667,27.83333333,22.83333333,21.33333333,23.91666667,22.33333333,17.66666667,14,11.5,9.333333333,8.583333333,6.416666667,3.916666667,5.833333333
3,Arizona,47.57446809,49.11702128,54.46808511,59.94680851,57.71276596,57.31914894,54.0212766,57.81914894,60.62765957,58.43617021,55.80851064,53.10638298,46.68085106,40.25531915,39.58510638,42.28723404,53.94680851,51.68085106,46.89361702,51.9787234,58.25531915,64.28723404,65.75531915,57.71276596,52.44680851,53.17021277,53.5106383,56,55.88297872,58.34042553,56.20212766,58,54.53191489,52.15957447,48.82978723,55.03191489,48.77659574,49.27659574,52.21276596,...,58.32978723,57.73404255,60.41489362,65.72340426,71.0212766,72.05319149,68.03191489,63.9893617,66.88297872,69.19148936,69.45744681,67.07446809,68.59574468,70.0106383,63.19148936,60.87234043,67.70212766,71.13829787,68.78723404,68.38297872,67.70212766,66.73404255,66.70212766,69.08510638,69.75531915,69.5,67.57446809,67.45744681,65.11702128,63.9787234,65.90425532,67.4893617,67.82978723,66.89361702,66.08510638,63.31914894,61.0212766,61.41489362,64.11702128,61.09574468
4,Arkansas,35.6,39.45,46.05,39,39.95,40.95,43.25,49.8,40.6,46.4,45.45,37.95,47.95,57.85,49.3,38.85,36.15,39.95,51.55,46.7,39.3,36.65,38.65,47.05,40.4,43.7,34.85,38.05,37.85,31.95,28.15,26.4,25.95,18.4,10.8,

In [ ]:
temp = np.array(temperature)
temp.shape

(51, 314)

In [ ]:
# remove column types
col_types = temp[0,:]
temp = temp[1:,:]
print(col_types)

['Date' '2021-01-12' '2021-01-13' '2021-01-14' '2021-01-15' '2021-01-16'
 '2021-01-17' '2021-01-18' '2021-01-19' '2021-01-20' '2021-01-21'
 '2021-01-22' '2021-01-23' '2021-01-24' '2021-01-25' '2021-01-26'
 '2021-01-27' '2021-01-28' '2021-01-29' '2021-01-30' '2021-01-31'
 '2021-02-01' '2021-02-02' '2021-02-03' '2021-02-04' '2021-02-05'
 '2021-02-06' '2021-02-07' '2021-02-08' '2021-02-09' '2021-02-10'
 '2021-02-11' '2021-02-12' '2021-02-13' '2021-02-14' '2021-02-15'
 '2021-02-16' '2021-02-17' '2021-02-18' '2021-02-19' '2021-02-20'
 '2021-02-21' '2021-02-22' '2021-02-23' '2021-02-24' '2021-02-25'
 '2021-02-26' '2021-02-27' '2021-02-28' '2021-03-01' '2021-03-02'
 '2021-03-03' '2021-03-04' '2021-03-05' '2021-03-06' '2021-03-07'
 '2021-03-08' '2021-03-09' '2021-03-10' '2021-03-11' '2021-03-12'
 '2021-03-13' '2021-03-14' '2021-03-15' '2021-03-16' '2021-03-17'
 '2021-03-18' '2021-03-19' '2021-03-20' '2021-03-21' '2021-03-22'
 '2021-03-23' '2021-03-24' '2021-03-25' '2021-03-26' '2021-03-27'
 '2

In [ ]:
#remove state names
states = temp[:,0]
temp = temp[:,1:]

In [ ]:
temp = np.expand_dims(temp, axis=2)
data = np.append(data,temp,axis=2)

In [ ]:
#temperature is now the last column
data.shape

(50, 313, 15)

# Save our data to google drive

In [ ]:
#change location of where the npy will be saved
training = data[:,0:233,:]
np.save('/content/drive/MyDrive/CG-ODE-main/CG-ODE-main/data/Dec/train_cs245.npy',training) #change location as needed

In [ ]:
#change location of where the npy will be saved
testing = data[:,233:233+31,:]
np.save('/content/drive/MyDrive/CG-ODE-main/CG-ODE-main/data/Dec/test_cs245.npy',testing)